In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    DDD_Train_Decile9 = pd.read_pickle("DDD_Train_Decile9.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile9.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile9, DDD_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 9')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

2264/2264 [==============================] - 12s 5ms/step - loss: 0.3294 - acc: 0.8737 - val_loss: 0.2447 - val_acc: 0.9171
Epoch 56/75
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3265 - acc: 0.8754 - val_loss: 0.2448 - val_acc: 0.9171
Epoch 57/75
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3301 - acc: 0.8693 - val_loss: 0.2445 - val_acc: 0.9171
Epoch 58/75
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3208 - acc: 0.8807 - val_loss: 0.2441 - val_acc: 0.9153
Epoch 59/75
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3200 - acc: 0.8759 - val_loss: 0.2408 - val_acc: 0.9153
Epoch 60/75
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3254 - acc: 0.8746 - val_loss: 0.2393 - val_acc: 0.9118
Epoch 61/75
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3260 - acc: 0.8666 - val_loss: 0.2416 - val_acc: 0.9083
Epoch 62/75
2264/2264 [==============================] - 12s

2264/2264 [==============================] - 12s 5ms/step - loss: 0.2900 - acc: 0.8821 - val_loss: 0.2253 - val_acc: 0.9118
Epoch 38/75
2264/2264 [==============================] - 12s 6ms/step - loss: 0.2906 - acc: 0.8777 - val_loss: 0.2249 - val_acc: 0.9153
Epoch 39/75
2264/2264 [==============================] - 12s 5ms/step - loss: 0.2895 - acc: 0.8843 - val_loss: 0.2253 - val_acc: 0.9136
Epoch 40/75
2264/2264 [==============================] - 12s 5ms/step - loss: 0.2815 - acc: 0.8825 - val_loss: 0.2253 - val_acc: 0.9118
Epoch 41/75
2264/2264 [==============================] - 12s 5ms/step - loss: 0.2908 - acc: 0.8812 - val_loss: 0.2252 - val_acc: 0.9118
Epoch 42/75
2264/2264 [==============================] - 12s 5ms/step - loss: 0.2887 - acc: 0.8754 - val_loss: 0.2252 - val_acc: 0.9118
Epoch 43/75
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2910 - acc: 0.8825 - val_loss: 0.2251 - val_acc: 0.9136
Epoch 44/75
2264/2264 [==============================] - 13s

2264/2264 [==============================] - 12s 5ms/step - loss: 0.3071 - acc: 0.8741 - val_loss: 0.2253 - val_acc: 0.9189
Epoch 22/100
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3045 - acc: 0.8799 - val_loss: 0.2242 - val_acc: 0.9171
Epoch 23/100
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3037 - acc: 0.8772 - val_loss: 0.2257 - val_acc: 0.9206
Epoch 24/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3079 - acc: 0.8737 - val_loss: 0.2316 - val_acc: 0.9171
Epoch 25/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2971 - acc: 0.8768 - val_loss: 0.2309 - val_acc: 0.9118
Epoch 26/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3086 - acc: 0.8719 - val_loss: 0.2338 - val_acc: 0.9171
Epoch 27/100
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3059 - acc: 0.8750 - val_loss: 0.2247 - val_acc: 0.9206

Epoch 00027: ReduceLROnPlateau reducing learning rate

Epoch 80/100
2264/2264 [==============================] - 14s 6ms/step - loss: 0.3072 - acc: 0.8737 - val_loss: 0.2253 - val_acc: 0.9136
Epoch 81/100
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2927 - acc: 0.8746 - val_loss: 0.2254 - val_acc: 0.9136
Epoch 82/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2936 - acc: 0.8785 - val_loss: 0.2253 - val_acc: 0.9136
Epoch 83/100
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2917 - acc: 0.8785 - val_loss: 0.2254 - val_acc: 0.9136
Epoch 84/100
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2983 - acc: 0.8807 - val_loss: 0.2252 - val_acc: 0.9136
Epoch 85/100
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2987 - acc: 0.8706 - val_loss: 0.2254 - val_acc: 0.9136
Epoch 86/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2911 - acc: 0.8834 - val_loss: 0.2258 - val_acc: 0.9136
Epoch 87/100
2264/2264 [=================

...

2264/2264 [==============================] - 13s 6ms/step - loss: 0.5417 - acc: 0.8136 - val_loss: 0.5056 - val_acc: 0.8254
Epoch 7/125
2264/2264 [==============================] - 13s 6ms/step - loss: 0.4942 - acc: 0.8136 - val_loss: 0.4495 - val_acc: 0.8272
Epoch 8/125
2264/2264 [==============================] - 13s 6ms/step - loss: 0.4634 - acc: 0.8140 - val_loss: 0.4162 - val_acc: 0.8342
Epoch 9/125
2264/2264 [==============================] - 13s 6ms/step - loss: 0.4486 - acc: 0.8242 - val_loss: 0.4026 - val_acc: 0.8554
Epoch 10/125
2264/2264 [==============================] - 12s 6ms/step - loss: 0.4411 - acc: 0.8432 - val_loss: 0.3900 - val_acc: 0.8589
Epoch 11/125
2264/2264 [==============================] - 13s 6ms/step - loss: 0.4398 - acc: 0.8308 - val_loss: 0.3813 - val_acc: 0.8713
Epoch 12/125
2264/2264 [==============================] - 12s 5ms/step - loss: 0.4351 - acc: 0.8397 - val_loss: 0.3760 - val_acc: 0.8713
Epoch 13/125
2264/2264 [==============================] -

Epoch 66/125
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3644 - acc: 0.8569 - val_loss: 0.2709 - val_acc: 0.9118
Epoch 67/125
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3548 - acc: 0.8604 - val_loss: 0.2717 - val_acc: 0.9101
Epoch 68/125
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3499 - acc: 0.8609 - val_loss: 0.2699 - val_acc: 0.9118
Epoch 69/125
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3466 - acc: 0.8635 - val_loss: 0.2678 - val_acc: 0.9136
Epoch 70/125
2264/2264 [==============================] - 14s 6ms/step - loss: 0.3500 - acc: 0.8626 - val_loss: 0.2655 - val_acc: 0.9153
Epoch 71/125
2264/2264 [==============================] - 15s 7ms/step - loss: 0.3519 - acc: 0.8697 - val_loss: 0.2637 - val_acc: 0.9171
Epoch 72/125
2264/2264 [==============================] - 14s 6ms/step - loss: 0.3457 - acc: 0.8684 - val_loss: 0.2631 - val_acc: 0.9171
Epoch 73/125
2264/2264 [=================

2264/2264 [==============================] - 11s 5ms/step - loss: 0.3048 - acc: 0.8741 - val_loss: 0.2414 - val_acc: 0.9153
Test accuracy: 0.915343914188
Train on 2264 samples, validate on 567 samples
Epoch 1/100
2264/2264 [==============================] - 18s 8ms/step - loss: 0.5897 - acc: 0.7041 - val_loss: 0.4417 - val_acc: 0.8219
Epoch 2/100
2264/2264 [==============================] - 14s 6ms/step - loss: 0.4221 - acc: 0.8273 - val_loss: 0.3299 - val_acc: 0.8748
Epoch 3/100
2264/2264 [==============================] - 14s 6ms/step - loss: 0.4025 - acc: 0.8344 - val_loss: 0.3106 - val_acc: 0.8818
Epoch 4/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3787 - acc: 0.8529 - val_loss: 0.2839 - val_acc: 0.8977
Epoch 5/100
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3671 - acc: 0.8582 - val_loss: 0.2676 - val_acc: 0.9065
Epoch 6/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3525 - acc: 0.8617 - val_loss: 0.2758 -

Epoch 67/100
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2977 - acc: 0.8728 - val_loss: 0.2251 - val_acc: 0.9136
Epoch 68/100
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2954 - acc: 0.8763 - val_loss: 0.2250 - val_acc: 0.9136
Epoch 69/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2915 - acc: 0.8816 - val_loss: 0.2253 - val_acc: 0.9136
Epoch 70/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2886 - acc: 0.8812 - val_loss: 0.2251 - val_acc: 0.9118
Epoch 71/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3026 - acc: 0.8807 - val_loss: 0.2254 - val_acc: 0.9136
Epoch 72/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2982 - acc: 0.8838 - val_loss: 0.2258 - val_acc: 0.9136
Epoch 73/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2965 - acc: 0.8737 - val_loss: 0.2259 - val_acc: 0.9118
Epoch 74/100
2264/2264 [=================

2264/2264 [==============================] - 13s 6ms/step - loss: 0.3021 - acc: 0.8777 - val_loss: 0.2328 - val_acc: 0.9118
Epoch 35/50
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2914 - acc: 0.8812 - val_loss: 0.2321 - val_acc: 0.9101
Epoch 36/50
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2940 - acc: 0.8816 - val_loss: 0.2320 - val_acc: 0.9101
Epoch 37/50
2264/2264 [==============================] - 12s 5ms/step - loss: 0.2881 - acc: 0.8816 - val_loss: 0.2320 - val_acc: 0.9118
Epoch 38/50
2264/2264 [==============================] - 12s 5ms/step - loss: 0.2842 - acc: 0.8754 - val_loss: 0.2321 - val_acc: 0.9118
Epoch 39/50
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2956 - acc: 0.8794 - val_loss: 0.2322 - val_acc: 0.9118
Epoch 40/50
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2916 - acc: 0.8821 - val_loss: 0.2319 - val_acc: 0.9101
Epoch 41/50
2264/2264 [==============================] - 14s

In [ ]:
0.86434